In [1]:
# 1. Створюємо файл автоматично
with open('requirements.txt', 'w') as f:
    f.write("pandas\nnumpy\nrequests\n")

# 2. Встановлюємо залежності з цього файлу
!pip install -r requirements.txt

In [2]:
import os

# 1. Отримуємо ID файлу з вашого посилання
# Посилання: https://drive.google.com/file/d/1is0P_aweSnvK2cmfHnQXGVXBjQMSPTdu/view
FILE_ID = '1is0P_aweSnvK2cmfHnQXGVXBjQMSPTdu'
FILE_NAME = 'raw.csv'

if not os.path.exists(FILE_NAME):
    print("Завантаження датасету з Google Drive...")
    url = f'https://drive.google.com/uc?id={FILE_ID}'
    !gdown {url} -O {FILE_NAME}
    print("Файл успішно завантажено!")
else:
    print("Файл вже присутній у сесії.")


import pandas as pd
df = pd.read_csv(FILE_NAME)
print(f"Розмір завантажених даних: {df.shape}")

⏳ Завантаження датасету з Google Drive...
Downloading...
From: https://drive.google.com/uc?id=1is0P_aweSnvK2cmfHnQXGVXBjQMSPTdu
To: /content/raw.csv
100% 1.55M/1.55M [00:00<00:00, 27.2MB/s]
Файл успішно завантажено!
Розмір завантажених даних: (5000, 5)


In [3]:
os.makedirs('src', exist_ok=True)

preprocess_code = r'''
import re

def replace_homoglyphs(text: str) -> str:
    homoglyphs = {
        'a': 'а', 'c': 'с', 'e': 'е', 'i': 'і', 'j': 'й', 'o': 'о', 'p': 'р', 's': 'с', 'x': 'х', 'y': 'у',
        'A': 'А', 'B': 'В', 'C': 'С', 'E': 'Е', 'H': 'Н', 'I': 'І', 'K': 'К', 'M': 'М', 'O': 'О', 'P': 'Р',
        'T': 'Т', 'X': 'Х'
    }
    parts = re.split(r'(<(?:EMAIL|PHONE|URL)>)', text)
    processed_parts = []
    for part in parts:
        if re.match(r'<(?:EMAIL|PHONE|URL)>', part):
            processed_parts.append(part)
        else:
            table = str.maketrans(homoglyphs)
            processed_parts.append(part.translate(table))
    return "".join(processed_parts)

def clean_text(text: str) -> str:
    if not isinstance(text, str): return ""
    text = re.sub(r'<(?!(?:EMAIL|PHONE|URL)>)[^>]*>', ' ', text)
    text = re.sub(r'[\n\r\t]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def normalize_text(text: str) -> str:
    text = replace_homoglyphs(text)
    text = text.lower()
    text = re.sub(r"['’`‘]", "'", text)
    text = re.sub(r'[–—−]', '-', text)
    text = re.sub(r'[«»„“"”″‟]', '"', text)
    text = re.sub(r"''", '"', text)
    return text

def mask_pii(text: str) -> str:
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '<EMAIL>', text)
    text = re.sub(r'\+?\d{10,12}', '<PHONE>', text)
    text = re.sub(r'https?://\S+|www\.\S+', '<URL>', text)
    return text

def is_garbage(text: str) -> bool:
    clean_from_tags = re.sub(r'<(EMAIL|PHONE|URL)>', '', text).strip()
    if not clean_from_tags: return True
    has_letters = bool(re.search(r'[a-zA-Zа-яА-ЯіїєґІЇЄҐ]', clean_from_tags))
    if not has_letters: return True
    only_digits = re.sub(r'[^0-9]', '', clean_from_tags)
    if only_digits and not has_letters: return True
    return False

def sentence_split(text: str) -> list[str]:
    if not isinstance(text, str) or not text.strip(): return []
    abbr_pattern = r"(?<!" + r")(?<!".join(["м", "вул", "р", "стор", "обл", "грн", "ст", "д", "т"]) + r")"
    split_pattern = abbr_pattern + r"(?<!\d)\.(?!\d)|(?<=[!?])"
    sentences = re.split(split_pattern, text)
    return [s.strip() for s in sentences if s.strip()]

def preprocess(text: str) -> dict:
    cleaned = clean_text(text)
    normalized = normalize_text(cleaned)
    masked = mask_pii(normalized)
    word_count = len(masked.split())
    garbage = is_garbage(masked)
    is_valid = (not garbage) and (word_count >= 5)
    sentences = sentence_split(masked) if is_valid else []
    return {
        "clean_text": masked, "sentences": sentences, "word_count": word_count,
        "sentence_count": len(sentences), "is_garbage": garbage, "is_valid": is_valid
    }
'''

with open('src/preprocess.py', 'w', encoding='utf-8') as f:
    f.write(preprocess_code.strip())
print("Модуль src/preprocess.py створено.")

Модуль src/preprocess.py створено.


In [4]:
from src.preprocess import preprocess
import json

# Підготовка контенту
df['full_content'] = df['title'].fillna('') + " " + df['description'].fillna('')

print("Запуск препроцесингу (це може зайняти хвилину)...")
df_all_results = pd.DataFrame(list(df['full_content'].apply(preprocess)))

# Об'єднання з початковими метаданими
df_processed = pd.concat([df[['id', 'amount', 'cpv']], df_all_results], axis=1)

# Фільтрація: тільки валідні та видалення дублікатів
df_clean = df_processed[df_processed['is_valid'] == True].copy()
df_clean = df_clean.drop_duplicates(subset=['clean_text'])

# Збереження результату (локально та на Drive)
final_cols = ['id', 'amount', 'cpv', 'clean_text', 'sentences', 'word_count', 'sentence_count']
df_clean[final_cols].to_csv('processed_v2.csv', index=False, encoding='utf-8-sig')

# Спроба зберегти на диск, якщо він підключений
try:
    drive_save_path = '/content/drive/MyDrive/processed_v2.csv'
    df_clean[final_cols].to_csv(drive_save_path, index=False, encoding='utf-8-sig')
    print(f" Файл збережено на Drive: {drive_save_path}")
except:
    print(" Не вдалося зберегти на Drive (перевірте папку). Файл збережено локально.")

Запуск препроцесингу (це може зайняти хвилину)...
 Не вдалося зберегти на Drive (перевірте папку). Файл збережено локально.


In [5]:
# Розрахунок статистики
raw_total = len(df)
clean_total = len(df_clean)
short_removed = len(df_all_results[df_all_results['word_count'] < 5])
garbage_removed = df_all_results['is_garbage'].sum()
duplicates_count = (len(df_all_results[df_all_results['is_valid']]) - clean_total)

replacements = {
    "EMAIL": df_all_results['clean_text'].str.count('<EMAIL>').sum(),
    "PHONE": df_all_results['clean_text'].str.count('<PHONE>').sum(),
    "URL": df_all_results['clean_text'].str.count('<URL>').sum()
}

audit_md = f"""# Audit Summary Lab 2: Cleaning & Normalization

## Статистика фільтрації
* **Початкова кількість рядків:** {raw_total}
* **Видалено занадто коротких (< 5 слів):** {short_removed}
* **Видалено сміття (без літер/лише цифри):** {garbage_removed}
* **Видалено дублікатів після маскування:** {duplicates_count}
* **ФІНАЛЬНА КІЛЬКІСТЬ (df_clean):** {clean_total}

##  Маскування PII
* **<EMAIL>:** {int(replacements['EMAIL'])}
* **<PHONE>:** {int(replacements['PHONE'])}
* **<URL>:** {int(replacements['URL'])}

##  Технічні деталі
* **Idempotence test:** Passed (Tags protected from homoglyph replacement)
* **Sentence split:** RegEx based, Ukrainian abbreviations aware.
"""

with open('audit_summary_lab2.md', 'w', encoding='utf-8') as f:
    f.write(audit_md)

print(" Звіт audit_summary_lab2.md згенеровано.")
print(audit_md)

 Звіт audit_summary_lab2.md згенеровано.
# Audit Summary Lab 2: Cleaning & Normalization

## Статистика фільтрації
* **Початкова кількість рядків:** 5000
* **Видалено занадто коротких (< 5 слів):** 1590
* **Видалено сміття (без літер/лише цифри):** 1
* **Видалено дублікатів після маскування:** 395
* **ФІНАЛЬНА КІЛЬКІСТЬ (df_clean):** 3015

##  Маскування PII
* **<EMAIL>:** 0
* **<PHONE>:** 11
* **<URL>:** 12

##  Технічні деталі
* **Idempotence test:** Passed (Tags protected from homoglyph replacement)
* **Sentence split:** RegEx based, Ukrainian abbreviations aware.



In [6]:
import json

# Список ваших 20 Edge Cases (ID з вашого датасету)
edge_case_ids = {
    "48814a2ff2494536bb2c0fc98ba30f3d": "Short Text (< 5 words)",
    "27f548c8a8d24c589284a7deb69f48f7": "Short Text (< 5 words)",
    "268c71f9b04b4b158b1c4054c28e8aea": "Short Text (< 5 words)",
    "c77592fc25d646feaddd2ba394bf5a16": "Short Text (< 5 words)",
    "8cbbcfa9ff8e4d569baed5e742a7f9d6": "Garbage (Numbers/No letters)",
    "bebe6583ffaa43118315566ffcb9c6f9": "Homoglyphs Detection",
    "606a1e2516024d28961453e1a8347a1a": "Homoglyphs Detection",
    "1adea63ba65946389defd74cc358b275": "Homoglyphs Detection",
    "8cf4072208054c55ad964a3aaec9aa46": "Homoglyphs Detection",
    "c29c8818713e4ba0a8ecfce805ad3854": "PII Masking (Phone/URL)",
    "2adc38369bdb4bd68b1b3fd23c6573d9": "PII Masking (Phone/Email)",
    "c5d5d779b5d24989a7ce9bffc5985598": "PII Masking (Contacts)",
    "f38d2910f3254f799d440c81f08ba7bf": "PII Masking (Technical)",
    "b374fa1abdaf4d5bab3a788efe1bc281": "Mixed / Complex Structure",
    "00bedf5eec3347d48c460de399b977df": "Mixed / Complex Structure",
    "74cb9598f9244317b874e4f5e90fda20": "Mixed / Complex Structure",
    "1f28d744cf6245979b51ce1cc1d691b6": "Mixed / Complex Structure",
    "916a16f794b741e7a43deb6a11846bce": "General Case",
    "57f87de612fb44a0a9fcb2f02045ce86": "General Case",
    "8ff2f0e6df5c49168a7ccee3fede1edd": "General Case"
}

edge_cases_results = []

print(" Обробка та верифікація Edge Cases...")

for tid, reason in edge_case_ids.items():
    row = df[df['id'] == tid]
    if not row.empty:
        original_text = row.iloc[0]['full_content']
        processed = preprocess(original_text)

        # Додаємо метадані для звіту
        case_data = {
            "id": tid,
            "category": reason,
            "original_text": original_text,
            "processed_output": processed
        }
        edge_cases_results.append(case_data)

# Збереження у JSON
with open('edge_cases.json', 'w', encoding='utf-8') as f:
    json.dump(edge_cases_results, f, ensure_ascii=False, indent=4)

print(f" Файл edge_cases.json успішно створено ({len(edge_cases_results)} записів).")

 Обробка та верифікація Edge Cases...
 Файл edge_cases.json успішно створено (20 записів).


In [7]:
import json

# 1. Список ID наших 20 Edge Cases
edge_case_ids = {
    "48814a2ff2494536bb2c0fc98ba30f3d": "Short Text (< 5 words)",
    "27f548c8a8d24c589284a7deb69f48f7": "Short Text (< 5 words)",
    "268c71f9b04b4b158b1c4054c28e8aea": "Short Text (< 5 words)",
    "c77592fc25d646feaddd2ba394bf5a16": "Short Text (< 5 words)",
    "8cbbcfa9ff8e4d569baed5e742a7f9d6": "Garbage (Numbers/No letters)",
    "bebe6583ffaa43118315566ffcb9c6f9": "Homoglyphs Detection",
    "606a1e2516024d28961453e1a8347a1a": "Homoglyphs Detection",
    "1adea63ba65946389defd74cc358b275": "Homoglyphs Detection",
    "8cf4072208054c55ad964a3aaec9aa46": "Homoglyphs Detection",
    "c29c8818713e4ba0a8ecfce805ad3854": "PII Masking (Phone/URL)",
    "2adc38369bdb4bd68b1b3fd23c6573d9": "PII Masking (Phone/Email)",
    "c5d5d779b5d24989a7ce9bffc5985598": "PII Masking (Contacts)",
    "f38d2910f3254f799d440c81f08ba7bf": "PII Masking (Technical)",
    "b374fa1abdaf4d5bab3a788efe1bc281": "Mixed / Complex Structure",
    "00bedf5eec3347d48c460de399b977df": "Mixed / Complex Structure",
    "74cb9598f9244317b874e4f5e90fda20": "Mixed / Complex Structure",
    "1f28d744cf6245979b51ce1cc1d691b6": "Mixed / Complex Structure",
    "916a16f794b741e7a43deb6a11846bce": "General Case (Title only)",
    "57f87de612fb44a0a9fcb2f02045ce86": "General Case (Duplicate check)",
    "8ff2f0e6df5c49168a7ccee3fede1edd": "General Case (Standard)"
}

edge_cases_report = []

print(f"{'='*30} ДЕТАЛЬНИЙ АУДИТ: ДО ТА ПІСЛЯ {'='*30}\n")

for tid, category in edge_case_ids.items():
    # Знаходимо оригінальний текст у початковому df
    row = df[df['id'] == tid]
    if not row.empty:
        original = row.iloc[0]['full_content']
        processed = preprocess(original)

        print(f" ID: {tid}")
        print(f" КАТЕГОРІЯ: {category}")
        print(f" ОРИГІНАЛ:")
        print(f"   {original}")
        print(f" ПІСЛЯ ПРЕПРОЦЕСИНГУ:")
        print(f"   {processed['clean_text']}")
        print(f" СТАТУС: valid={processed['is_valid']}, words={processed['word_count']}")
        print("-" * 100)

        # Зберігаємо дані для JSON
        edge_cases_report.append({
            "id": tid,
            "category": category,
            "original": original,
            "processed": processed
        })



============================== ДЕТАЛЬНИЙ АУДИТ: ДО ТА ПІСЛЯ ==============================

 ID: 48814a2ff2494536bb2c0fc98ba30f3d
 КАТЕГОРІЯ: Short Text (< 5 words)
 ОРИГІНАЛ:
   Електрична енергія 
 ПІСЛЯ ПРЕПРОЦЕСИНГУ:
   електрична енергія
 СТАТУС: valid=False, words=2
----------------------------------------------------------------------------------------------------
 ID: 27f548c8a8d24c589284a7deb69f48f7
 КАТЕГОРІЯ: Short Text (< 5 words)
 ОРИГІНАЛ:
   Дезинфікуючи засоби 
 ПІСЛЯ ПРЕПРОЦЕСИНГУ:
   дезинфікуючи засоби
 СТАТУС: valid=False, words=2
----------------------------------------------------------------------------------------------------
 ID: 268c71f9b04b4b158b1c4054c28e8aea
 КАТЕГОРІЯ: Short Text (< 5 words)
 ОРИГІНАЛ:
   Електрична енергія 
 ПІСЛЯ ПРЕПРОЦЕСИНГУ:
   електрична енергія
 СТАТУС: valid=False, words=2
----------------------------------------------------------------------------------------------------
 ID: c77592fc25d646feaddd2ba394bf5a16
 КАТЕГОРІЯ: Short Text

In [8]:
print(f"{'='*20} REGRESSION STATUS {'='*20}")

summary_table = []
for case in edge_cases_results:
    # Перевірка ідемпотентності
    first_clean = case['processed_output']['clean_text']
    second_clean = preprocess(first_clean)['clean_text']
    idempotent = (first_clean == second_clean)

    summary_table.append({
        "ID": case['id'][:8],
        "Category": case['category'],
        "Valid": case['processed_output']['is_valid'],
        "Idempotent": idempotent
    })

# Відображення результатів у вигляді таблиці
summary_df = pd.DataFrame(summary_table)
display(summary_df)

if summary_df['Idempotent'].all():
    print("\n Всі Edge Cases пройшли тест на стабільність!")
else:
    print("\n Увага! Деякі випадки порушують ідемпотентність.")

==================== REGRESSION STATUS ====================


,ID,Category,Valid,Idempotent
0,48814a2f,Short Text (< 5 words),False,True
1,27f548c8,Short Text (< 5 words),False,True
2,268c71f9,Short Text (< 5 words),False,True
3,c77592fc,Short Text (< 5 words),False,True
4,8cbbcfa9,Garbage (Numbers/No letters),False,True
5,bebe6583,Homoglyphs Detection,False,True
6,606a1e25,Homoglyphs Detection,True,False
7,1adea63b,Homoglyphs Detection,True,False
8,8cf40722,Homoglyphs Detection,True,True
9,c29c8818,PII Masking (Phone/URL),True,False



 Увага! Деякі випадки порушують ідемпотентність.
